In [1]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 
%matplotlib inline

* Well — идентификатор скважины;
* Por — пористость скважины (%);
* Perm — проницаемость скважины;
* AI — акустический импеданс
* Brittle — коэффициент хрупкости скважины (%);
* TOC — общий органический углерод (%);
* VR — коэффициент отражения витринита (%);
* Prod — добыча газа в сутки (млн. кубических футов).

In [2]:
df=pd.read_csv('data/unconv.csv')

In [3]:
Cor=df.corr()

In [4]:
np.linalg.matrix_rank(Cor)

8

In [5]:
round(np.linalg.det(Cor), 4)

0.0007

In [6]:
X1=df.drop(['Prod'], axis=1)
X=np.column_stack((np.ones(200), X1))
y=df[['Prod']]

In [7]:
w_hat=np.linalg.inv(X.T@X)@X.T@y

In [8]:
Cor

,Well,Por,Perm,AI,Brittle,TOC,VR,Prod
Well,1.000000,0.068927,0.077928,0.041483,-0.079252,0.022624,-0.007279,0.026817
Por,0.068927,1.000000,0.760546,-0.461549,-0.218570,0.711831,0.111860,0.861910
Perm,0.077928,0.760546,1.000000,-0.239636,-0.124017,0.471746,0.051023,0.727426
AI,0.041483,-0.461549,-0.239636,1.000000,0.127599,-0.531864,0.499143,-0.390835
Brittle,-0.079252,-0.218570,-0.124017,0.127599,1.000000,-0.214282,0.317929,0.237155
TOC,0.022624,0.711831,0.471746,-0.531864,-0.214282,1.000000,0.299483,0.654445
VR,-0.007279,0.111860,0.051023,0.499143,0.317929,0.299483,1.000000,0.323182
Prod,0.026817,0.861910,0.727426,-0.390835,0.237155,0.654445,0.323182,1.000000


In [9]:
w_hat

,Prod
0,-1232.308030
1,0.050700
2,230.179140
3,116.239006
4,-365.202301
5,24.994370
6,-78.400929
7,785.259815


In [10]:
print (w_hat.values[0])

[-1232.30802956]


In [11]:
x_new = np.array([1, 106, 15.32, 3.71, 3.29, 55.99, 1.35, 2.42])
y_new = 4748.315024
y_new_pred = x_new @ w_hat
print(np.round(np.abs(y_new_pred - y_new), 0))

Prod    25.0
dtype: float64


In [12]:
from sklearn.metrics import mean_absolute_percentage_error
y_pred = X @ w_hat
print('Result MAPE {:.1f}:'.format(mean_absolute_percentage_error(y, y_pred)*100))

Result MAPE 3.6:


In [13]:
from sklearn.metrics import mean_absolute_percentage_error
X = df.drop(['Prod', 'Perm', 'TOC', 'Well'], axis=1)
y = df['Prod'].values
index = ['intercept']+list(X.columns)
n = X.shape[0]
X = np.column_stack((np.ones(n), X))
w_hat = np.linalg.inv(X.T@X)@X.T@y
y_pred = X @ w_hat
print(pd.Series(np.round(w_hat, 0), index=index))
print(f'MAPE: {mean_absolute_percentage_error(y, y_pred)*100:.1f} %')

intercept   -1835.0
Por           293.0
AI           -200.0
Brittle        28.0
VR            517.0
dtype: float64
MAPE: 4.0 %


In [14]:
X_new=df[['Por', 'AI', 'Brittle', 'VR']]
X_new1=np.column_stack((np.ones(200), X_new))
y=df['Prod']

In [15]:
w_hat_new=np.linalg.inv(X_new1.T@X_new1)@X_new1.T@y

In [16]:
w_hat_new

array([-1835.44646069,   293.03624565,  -200.03091206,    27.64098209,
         517.40272597])

In [17]:
from sklearn.preprocessing import PolynomialFeatures

In [38]:
poly=PolynomialFeatures(degree=3, include_bias=False)
X_new_poly=poly.fit_transform(X_new)

In [24]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate

lin_reg=LinearRegression()
lin_reg.fit(X_new_poly, y)
y_new_pred=lin_reg.predict(X_new_poly)

cv_result=cross_validate(lin_reg, X_new_poly, y, scoring='neg_mean_absolute_percentage_error', cv=5, return_train_score=True)

print('MAPE на тренировочных фолдах: {:.1f} %'.format(-cv_result['train_score'].mean()* 100))
print('MAPE на валидационных фолдах: {:.1f} %'.format(-cv_result['test_score'].mean() * 100))	

MAPE на тренировочных фолдах: 1.8 %
MAPE на валидационных фолдах: 2.7 %


In [25]:
from sklearn.preprocessing import StandardScaler

In [39]:
scaler=StandardScaler()
X_new_poly=scaler.fit_transform(X_new_poly)

In [40]:
from sklearn.linear_model import Lasso

In [42]:
scaler=StandardScaler()
X_new=scaler.fit_transform(X_new)

poly=PolynomialFeatures(degree=3, include_bias=False)

X_new_poly=poly.fit_transform(X_new)



l1=Lasso(alpha=5)
l1.fit(X_new_poly, y)
# оцениваем качество модели на кросс-валидации
cv_results = cross_validate(l1, X_new_poly, y, scoring='neg_mean_absolute_percentage_error', cv=5, return_train_score=True)
print('MAPE на тренировочных фолдах: {:.1f} %'.format(-cv_results['train_score'].mean()* 100))
print('MAPE на валидационных фолдах: {:.1f} %'.format(-cv_results['test_score'].mean() * 100))


MAPE на тренировочных фолдах: 1.8 %
MAPE на валидационных фолдах: 2.3 %


In [43]:
from sklearn.linear_model import Ridge

In [44]:
l2=Ridge(alpha=1)

l2.fit(X_new_poly, y)

# оцениваем качество модели на кросс-валидации
cv_results = cross_validate(l2, X_new_poly, y, scoring='neg_mean_absolute_percentage_error', cv=5, return_train_score=True)
print('MAPE на тренировочных фолдах: {:.1f} %'.format(-cv_results['train_score'].mean()* 100))
print('MAPE на валидационных фолдах: {:.1f} %'.format(-cv_results['test_score'].mean() * 100))

MAPE на тренировочных фолдах: 1.8 %
MAPE на валидационных фолдах: 2.7 %
